In [1]:
!pip install -r requirements.txt

  Using cached anyio-4.8.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached asttokens-3.0.0-py3-none-any.whl.metadata (4.7 kB)
  Using cached colorama-0.4.6-py2.py3-none-any.whl.metadata (17 kB)
  Using cached comm-0.2.2-py3-none-any.whl.metadata (3.7 kB)
  Using cached datasets-3.2.0-py3-none-any.whl.metadata (20 kB)
  Using cached debugpy-1.8.12-cp311-cp311-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.3 kB)
  Using cached decorator-5.1.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached executing-2.1.0-py2.py3-none-any.whl.metadata (8.9 kB)
  Using cached fsspec-2024.9.0-py3-none-any.whl.metadata (11 kB)
  Using cached httpx-0.27.2-py3-none-any.whl.metadata (7.1 kB)
  Using cached ipykernel-6.29.5-py3-none-any.whl.metadata (6.3 kB)
  Using cached ipython-8.31.0-py3-none-any.whl.metadata (4.9 kB)
  Using cached jedi-0.19.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cach

In [2]:
! python -m spacy download fr_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 56.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [3]:
with open("example_md_to_text.txt", "r", encoding="latin-1") as f: # TO DO: check proper encoding for .md files
    markdown_example = f.read()

In [4]:
from pprint import pprint
import tqdm
# pprint(markdown_example)

Basic OLlama Workflow

In [5]:
# import ollama

In [6]:
# # basic workflow of getting llama embeddings with ollama
# ollama.embeddings(model="llama3.2:3b",
#                 prompt="Hello")

In [7]:
# # basic workflow of generating responses with ollama
# ollama.generate(model="llama3.2:3b",
#                 prompt="Whos is Obama")

Loading HF to get Attention Masks

In [8]:
with open("HF_TOKEN.txt", "r") as f:
    hf_token = f.read()

In [9]:
from huggingface_hub import HfFolder, whoami

HfFolder.save_token(hf_token)
print(whoami()["name"])

alberto-lorente


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [10]:
import torch

device = "cpu"
if torch.cuda.is_available():
  print("Cuda available")
  device = "cuda:0"

Cuda available


In [11]:
# using huggingface tokenizer for attettion layer
# make sure you have GPU enabled
# takes around 5 mins to load with TPUs

from transformers import AutoTokenizer, AutoModel

model_id = "HIT-TMG/KaLM-embedding-multilingual-mini-v1" # which model to use?
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_id)
model = AutoModel.from_pretrained(model_id).to(device)

In [12]:
model

Qwen2Model(
  (embed_tokens): Embedding(151936, 896)
  (layers): ModuleList(
    (0-23): 24 x Qwen2DecoderLayer(
      (self_attn): Qwen2SdpaAttention(
        (q_proj): Linear(in_features=896, out_features=896, bias=True)
        (k_proj): Linear(in_features=896, out_features=128, bias=True)
        (v_proj): Linear(in_features=896, out_features=128, bias=True)
        (o_proj): Linear(in_features=896, out_features=896, bias=False)
        (rotary_emb): Qwen2RotaryEmbedding()
      )
      (mlp): Qwen2MLP(
        (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
        (up_proj): Linear(in_features=896, out_features=4864, bias=False)
        (down_proj): Linear(in_features=4864, out_features=896, bias=False)
        (act_fn): SiLU()
      )
      (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
    )
  )
  (norm): Qwen2RMSNorm((896,), eps=1e-06)
  (rotary_emb): Qwen2RotaryEmbedding()
)

- text enriching with VLLM to add a little description of tables before hey appear?

- embeddings that return attention layer

- perform similarity on the attention layer -> cluster similar sentences

- summarize the clusters

- tree-based graph

## Splitting the text into sentences

This method may take a lot of time. Alternatives?

In [13]:
import spacy

nlp = spacy.load("fr_core_news_sm")

In [14]:
doc = nlp(markdown_example)
sents = [sent.text for sent in doc.sents]

In [15]:
sents[0:10]

['# Séance publique\n\n\n\n# du jeudi 28 octobre 2021\n\n\n\n# à 18h00\n\n\n\n# Chorum',
 'Alain Gilles',
 '- Halle Vacheresse\n\n\n\n#',
 'Rue des Vernes à Roanne\n\n\n\n#',
 'PROCES',
 'VERBAL\n\n\n\n',
 "L'an deux mille vingt et un, le 28 octobre à 18 h 00, les conseillers communautaires de Roannais Agglomération, se sont réunis à l\x92Espace Chorum \x96 Halle",
 'Vacheresse \x96',
 'Rue des Vernes à Roanne.\n\n\n\n',
 'La convocation de tous les conseillers a été faite le 22 octobre 2021, dans les formes et délais prescrits par la loi, par Yves Nicolin, Président.\n\n\n\n']

## Getting the text embeddings


NOTE: There is one important caveat. We are computing sentence embeddings. If we were to pass the whole doc, the attention mask for each token would look very different.

### Example for one piece of text

In [16]:
# dir(tokenizer)

In [17]:
tokenized_markdown = tokenizer(markdown_example, return_tensors="pt").to(device)

In [18]:
len(tokenized_markdown[0])

20468

In [19]:
model.eval()
with torch.no_grad():
  # first batch (only one since we are processing one doc)
  # final token
    embeddings = model(**tokenized_markdown)[0][:, 0]

In [20]:
embeddings.shape

torch.Size([1, 896])

We take the last token since it's the one which contains all the aggregate info.

In [21]:
# embeddings

In [22]:
import torch.nn.functional as F

normalized_embeddings = F.normalize(embeddings, p=2, dim=1)

In [23]:
# normalized_embeddings

### For a list of sentences

In [24]:
def compute_embeddings(sentence):

    tokenized_sentences = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True).to(device)

    model.eval()
    with torch.no_grad():
        embeddings = model(**tokenized_sentences)[0][:, 0]
        # print(embeddings.shape)

    normalized_embeddings = F.normalize(embeddings, p=2, dim=1)
    detached_embeddings = normalized_embeddings.detach().cpu().numpy() # detached into cpu so that we can manipulate them for clustering

    torch.cuda.empty_cache() # Careful with running out of memory

    return detached_embeddings

In [25]:
detached_embeddings = compute_embeddings(sents[0])

In [26]:
detached_sents_embeddings = [compute_embeddings(sent) for sent in sents]

In [27]:
len(detached_sents_embeddings)

913

In [28]:
len(detached_sents_embeddings) == len(sents)

True

In [29]:
detached_sents_embeddings[0].shape

(1, 896)

## Clustering

In [30]:
unsqueezeded_embeddings = [sent[0] for sent in detached_sents_embeddings] # there is an extra useless dimension

In [31]:
# unsqueezeded_embeddings[0:2]

### Example for one number of clusters

In [32]:
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score

In [33]:
gm = GaussianMixture(n_components=4, random_state=42)
clusters = gm.fit_predict(unsqueezeded_embeddings)

In [34]:
# clusters # label assigned to each sentence

In [35]:
sil_sc = silhouette_score(unsqueezeded_embeddings, clusters)
print(sil_sc) # the closer to 1 the better (how similar is an object to its cluster compared to the other clusters)

0.43139583


### Optimal number of clusters

In [36]:
import numpy as np

range_clusters = np.arange(start=3, stop=9, step=1)
print(range_clusters)

[3 4 5 6 7 8]


We could compute jensenshannon distance as well.

In [37]:
def cluster_n(n_clusters, embeddings):

  gm = GaussianMixture(n_components=n_clusters, random_state=42)
  clusters = gm.fit_predict(embeddings)
  sil_sc = silhouette_score(unsqueezeded_embeddings, clusters)

  print("Number of clusters: ", n_clusters)
  print("Score: ", sil_sc)
  print()

  return clusters, sil_sc

In [38]:
silhouette_scores = []

for n_cluster in range_clusters:
  _, sil_sc = cluster_n(n_cluster, unsqueezeded_embeddings)
  silhouette_scores.append(sil_sc)

Number of clusters:  3
Score:  0.42344064

Number of clusters:  4
Score:  0.43139583

Number of clusters:  5
Score:  0.45841455

Number of clusters:  6
Score:  0.4844027

Number of clusters:  7
Score:  0.52773994

Number of clusters:  8
Score:  0.48493922



In [39]:
max = np.argmax(silhouette_scores)
optimal_n = range_clusters[max]
print("Index", max)
print("Optimal Number of Clusters", optimal_n)

Index 4
Optimal Number of Clusters 7


In [40]:
clusters, _ = cluster_n(optimal_n, unsqueezeded_embeddings)

Number of clusters:  7
Score:  0.52773994



In [41]:
# clusters

### Logging Information, concatinating the text relating to each cluster and recomputing embeddings

In [42]:
# sents[:4]

In [43]:
# clusters[:4]

In [44]:
# unsqueezeded_embeddings[:4]

In [45]:
# getting the indexes for the sentences belonging to each cluster

dict_clusters_indexes = {}
for cluster in range_clusters:
  dict_clusters_indexes[cluster] = [] # the key is the cluster id
  for index, sent_cluster in enumerate(clusters): # we are searching linearly so they should be sorted in ascending order already
    if sent_cluster == cluster:
      dict_clusters_indexes[cluster].append(index) # the list are the indexes that have that cluster id

In [46]:
# dict_clusters_indexes

So far we have:
- sents = the actual sentences split from SpaCy.
- unsqueezed_embeddings = the embeddings of the sentences detached and unsqueezed.
- clusters = the cluster each sentence belongs to.
- dict_clusters_indexes = the indexes of the sentences of each cluster

In [47]:
# sents
# unsqueezeded_embeddings
# clusters
# dict_clusters_indexes

In [48]:
sentence_info = {}
for index, sentence in enumerate(sents):

  key = f"sent_n_{index}"
  sentence_info[key] = {}
  sentence_info[key]["text"] = sentence
  sentence_info[key]["embedding"] = unsqueezeded_embeddings[index]
  sentence_info[key]["cluster"] = clusters[index]
  sentence_info[key]["index"] = index


In [49]:
# sentence_info['sent_n_0']

In [50]:
cluster_aggregates = {}
for cluster in range(optimal_n):

  # making a list of the dictionaries whose cluster matches the current cluster in the loop
  list_dicts_sents = [dict_sent for dict_sent in sentence_info.values() if dict_sent["cluster"] == cluster]

  # making list of the texts, embeddings, indexes of all the sentences belonging to a cluster
  list_texts = [dict_sent["text"] for dict_sent in list_dicts_sents]
  list_embeddings = [dict_sent["embedding"] for dict_sent in list_dicts_sents]
  list_indexes = [dict_sent["index"] for dict_sent in list_dicts_sents]

  cluster_aggregates[cluster] = {"texts": list_texts,
                                "embeddings": list_embeddings,
                                "indexes": list_indexes
                                }

In [51]:
cluster_aggregates.keys()

dict_keys([0, 1, 2, 3, 4, 5, 6])

In [52]:
# cluster_aggregates[5]["texts"][15:30]

Note: if we look at the text we can see that the clustering seems to group information of the same type but given that we got the embeddings for each sentence separately, computing global attention would probably leave us with better and more consistent sentence clusters. We can also augment the chunks by looking at start and end index of each sentence in the raw text and just adjusting a stride parameter. TO DO

### Sumarization

Now that we have the basic separation wrt to clusters, we can start creating different representations for each cluster and doing the hierarchical indexing.

In [53]:
from transformers import AutoModelForSeq2SeqLM

summ_model = 'csebuetnlp/mT5_multilingual_XLSum'

tokenizer_summ = AutoTokenizer.from_pretrained(summ_model)
model_summ = AutoModelForSeq2SeqLM.from_pretrained(summ_model).to(device)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [54]:
import re
WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip())) # was part of the docs

### Individual example

In [55]:
example_cluster = " ".join(cluster_aggregates[5]["texts"])
example_cluster

"PROCES Christine Aranéo Page 1 sur 49\n\n\n\n|Absents|Ni pouvoir|Suppléant|Pouvoir donné à\x85|Ni suppléant|\n\n|---|---|---|---|---|\n\n|Jean -Marc Ambroise|X| Boyer| |Yves Meunier| -Marc Detour| Dupuis| Approbation des procès-verbaux des conseils communautaires des 24 juin et 22 juillet 2021 ASSEMBLEES\n\n\n\n# 1. Agglomération a délégué au président et au bureau communautaire des attributions.\n\n\n\n 2021 316 du 17 septembre 2021 et entretien d\x92abris de bus et de mobiliers d\x92information d\x92Enseignement Supérieur en vue du regroupement des formations sur le Campus Mendes France à Roanne (Phase 2 : travaux de construction) avec la société ROCHE SARL ;\n\n 2021 319 du 20 septembre 2021 Abonnement FAS (Frais d\x92Accès au Service) 2021 322 du 21 septembre 2021 CASSE 44, dont le siège est situé 3 ? rue de la Barbotte partenaire de CIDER Engineering ;\n\n Agglomération ;\n\n 2021 323 du 22 septembre 2021 n° 1\n\n\n\n BoOm » Délibérations prises par le Président\n\n\n\n# N° DP 20

In [56]:
tokenized_example = tokenizer_summ(
                              [WHITESPACE_HANDLER(example_cluster)],
                              return_tensors="pt",
                              padding="max_length",
                              truncation=True
                          ).to(device)

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [57]:
tokenized_example

{'input_ids': tensor([[   259, 167667,    259,  ...,  25645,    263,      1]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]], device='cuda:0')}

In [58]:
input_ids = tokenized_example["input_ids"]
# input_ids = tokenized_example["input_ids"].squeeze(dim=0) NOP,expects the unsqueexed dimension

In [59]:
input_ids

tensor([[   259, 167667,    259,  ...,  25645,    263,      1]],
       device='cuda:0')

In [60]:
len(input_ids[0])

3030

In [61]:
output = model_summ.generate(
    input_ids=input_ids,
    max_length=1000,
    no_repeat_ngram_size=2,
    num_beams=4) # if it is too slow, adjust num_beams and max_len

In [62]:
output[0]

tensor([     0,   1760,   1888,  43159,    498,  23383,    263,    317, 162817,
          7636,    269,    283,  16674,    265,    269,   3897,  22770,    259,
           267,      1], device='cuda:0')

In [63]:
summary = tokenizer_summ.decode(
    output[0],
    skip_special_tokens=True,
    clean_up_tokenization_spaces=False
)

In [64]:
summary

'Les résultats des conseils communautaires de la commune de Roanne :'

### For the clusters

In [65]:
from torch.amp import autocast

In [70]:
def generate_summary(concat_text):

  WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))

  tokenized_example = tokenizer_summ(
                              [WHITESPACE_HANDLER(concat_text)],
                              return_tensors="pt",
                              padding="max_length",
                              truncation=True).to(device)

  input_ids = tokenized_example["input_ids"]

  model_summ.eval()
  with autocast("cuda"):
    summary_embedding_output = model_summ.generate(
                                              input_ids=input_ids,
                                              max_length=1000,
                                              no_repeat_ngram_size=2,
                                              num_beams=4) # if it is too slow, adjust num_beams and max_len

  summary_decoded = tokenizer_summ.decode(
                                    summary_embedding_output[0],
                                    skip_special_tokens=True,
                                    clean_up_tokenization_spaces=False)

  torch.cuda.empty_cache() # Careful with running out of memory

  return summary_embedding_output.detach().cpu().numpy(), summary_decoded

In [67]:
torch.cuda.empty_cache()

In [71]:
def process_cluster(cluster, cluster_aggregates=cluster_aggregates):

  print("Procesing cluster ", cluster+1)
  torch.cuda.empty_cache()

  cluster_aggregates[cluster]["broader_representations"] = {}

  concat_text = " ".join(cluster_aggregates[cluster]["texts"])
  cluster_aggregates[cluster]["broader_representations"]["concat_text"] = concat_text


  cluster_embeddings = compute_embeddings(concat_text)
  cluster_aggregates[cluster]["broader_representations"]["cluster_sentence_embeddings"] = cluster_embeddings[0] # taking out the extra unnecesary dimension

  torch.cuda.empty_cache()


  summary_embedding_output, summary_decoded = generate_summary(concat_text)
  cluster_aggregates[cluster]["broader_representations"]["cluster_summary_embedding"] = summary_embedding_output
  cluster_aggregates[cluster]["broader_representations"]["cluster_summary_decoded"] = summary_decoded

  return cluster_aggregates


In [72]:
for cluster in list(cluster_aggregates.keys()):
  cluster_aggregates = process_cluster(cluster)

Procesing cluster  1
Procesing cluster  2
Procesing cluster  3


OutOfMemoryError: CUDA out of memory. Tried to allocate 3.73 GiB. GPU 0 has a total capacity of 14.75 GiB of which 941.06 MiB is free. Process 181395 has 13.83 GiB memory in use. Of the allocated memory 13.52 GiB is allocated by PyTorch, and 183.59 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
cluster_aggregates[0]["broader_representations"].keys()